<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/resume_rag_system_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain.openai openai langchain-community langsmith chromadb python-dotenv sentence-transformers pypdf langchain_community langchain_experimental

## API keys from .env

In [ ]:
import os
from google.colab import drive
from dotenv import load_dotenv

drive.mount("/content/drive")

load_dotenv(dotenv_path="/content/drive/MyDrive/Projects/.env/.env")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

## Read pdf from drive
HOLD. will work with docs now, import PDF later.

## Create docs for "about me"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
# from sentence_transformers import SentenceTransformer # does not work well w langchain/chroma; use SentenceTransformerEmbeddings instead
from langchain_community.embeddings import SentenceTransformerEmbeddings

docs = [
    Document(page_content="Jack likes to build workflows and AI systems"),
    Document(page_content="Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith"),
    Document(page_content="Jack is friends with Tom and Sally."),
    Document(page_content="Jack loves to trek on weekends."),
    Document(page_content="Jack loves to watch movies and listen to music.")
]

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    embedding = embedding_model,
    documents = docs,
    collection_name = "random_db_2",
    persist_directory = "random_db_2"
)

# check if docs are added
added_docs = vectorstore.get()

print(f"added {len(added_docs)} docs")
print(added_docs)

/tmp/ipython-input-669439731.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

added 7 docs
{'ids': ['01b54da6-292b-4e98-9a18-4f385a412faa', '9d950ef6-cb08-4725-a22a-1d89f5c7f941', 'f73bda25-0740-4a0d-b0a7-d259455172f4', '44a1bec2-8144-4a31-8907-280c15a7f3d4', 'bee2ee8a-c733-409d-a2f8-c24cedb836c9', 'e2ca3122-5fdd-4837-b874-84f3c3aca296', '730dafa8-6d39-420f-8463-978812df0261', '2e3f0bd9-c31b-41df-acb1-0c02e2d338ed', '8412d523-f2b0-4d5a-9fad-624d88fe4984', '44e59ece-f25c-47a5-9593-2abe0174d23c'], 'embeddings': None, 'documents': ['Jack likes to build workflows and AI systems', 'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith', 'Jack is friends with Tom and Sally.', 'Jack loves to trek on weekends.', 'Jack loves to watch movies and listen to music.', 'Jack likes to build workflows and AI systems', 'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith', 'Jack is friends with Tom and Sally.', 'Jack loves to trek on weekends.', 'Jack loves to watch movies and listen to music.'], 'uris': None, 'included': ['

### Retrieve docs from VS based on query sim searches

In [ ]:
# delete chromadb to start from scratch
# vectorstore.delete_collection()

added = vectorstore.get()
added

{'ids': ['01b54da6-292b-4e98-9a18-4f385a412faa',
  '9d950ef6-cb08-4725-a22a-1d89f5c7f941',
  'f73bda25-0740-4a0d-b0a7-d259455172f4',
  '44a1bec2-8144-4a31-8907-280c15a7f3d4',
  'bee2ee8a-c733-409d-a2f8-c24cedb836c9',
  'e2ca3122-5fdd-4837-b874-84f3c3aca296',
  '730dafa8-6d39-420f-8463-978812df0261',
  '2e3f0bd9-c31b-41df-acb1-0c02e2d338ed',
  '8412d523-f2b0-4d5a-9fad-624d88fe4984',
  '44e59ece-f25c-47a5-9593-2abe0174d23c'],
 'embeddings': None,
 'documents': ['Jack likes to build workflows and AI systems',
  'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith',
  'Jack is friends with Tom and Sally.',
  'Jack loves to trek on weekends.',
  'Jack loves to watch movies and listen to music.',
  'Jack likes to build workflows and AI systems',
  'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith',
  'Jack is friends with Tom and Sally.',
  'Jack loves to trek on weekends.',
  'Jack loves to watch movies and listen to music.'],
 'u

# Search Strategies
- similiarty
- MMR
- context compression
- semantic chunker

In [ ]:
# queries
QUERY1 = "who are Jack's friends?"
QUERY2 = "what are jack's hobbies?"
QUERY3 = "what does jack work on?"

## Basic similarity search

In [ ]:
# basic sim search
basic_search = vectorstore.similarity_search(
    query = QUERY2,
    k = 2
)
for doc in basic_search:
  print(doc.page_content)

# Output
# vectorstore.similary adhers to k; does not work invoking runnable (k)

# Jack loves to watch movies and listen to music.
# Jack loves to trek on weekends.


Jack loves to watch movies and listen to music.
Jack loves to watch movies and listen to music.


## MMR

In [ ]:
# MMR
base_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs = { "k": 2 }
)
res = base_retriever.invoke(QUERY2)
for doc in res:
  print(doc.page_content)

# notes - k is ignored. why?; need to used search_kwargs, not "k"
# output
# Jack loves to watch movies and listen to music.
# Jack likes to build workflows and AI systems


Jack loves to watch movies and listen to music.
Jack likes to build workflows and AI systems


## contextual compression thingie

In [ ]:
# needs llm, embedding, compression mod, base retr, LLMChainExtractor
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    model="gpt-3.5-turbo")

base_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs = { "k": 2 }
)

base_compressor = LLMChainExtractor.from_llm(
    llm = model,
)

compressor_retriever = ContextualCompressionRetriever(
    base_retriever = base_retriever,
    base_compressor = base_compressor
)
result_docs = compressor_retriever.invoke(QUERY3)

# OUTPUTS - IMPRESSIVE!!

# QUERY1
# Jack is friends with Tom and Sally.

# QUERY2
# Jack loves to watch movies and listen to music.
# Jack likes to build workflows and AI systems

# QUERY3
# Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith

for doc in result_docs:
  print(doc.page_content)

Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith


## Load pdf from drive

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
file_path = "/content/drive/MyDrive/Projects/docs/rodi.pdf"

about_rodi_loader = PyPDFLoader(file_path)

about_pdf_docs = about_rodi_loader.load()
print(f"docs loaded = {len(docs)}")
print(docs[0].page_content)
doc = docs[0].page_content

Mounted at /content/drive
docs loaded = 1
High,  higher  than  the  sun  You  shoot  me  from  a  gun  I  need  you  to  elevate  me  here,  At  the  corner  of  your  lips  As  the  orbit  of  your  hips  Eclipse,  you  elevate  my  soul   I've  lost  all  self-control  Been  living  like  a  mole  Now  going  down,  excavation  I  and  I  in  the  sky  You  make  me  feel  like  I  can  fly  So  high,  elevation   A  star  lit  up  like  a  cigar  Strung  out  like  a  guitar  Maybe  you  could  educate  my  mind  Explain  all  these  controls  I  can't  sing  but  I've  got  soul  The  goal  is  elevation   A  mole,  living  in  a  hole


## Simple recursive text splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

chunks = splitter.split_text(doc)
print(chunks)
for doc in chunks:
  print(doc)
len(chunks)



['High,  higher  than  the  sun  You  shoot  me  from  a  gun  I  need  you  to  elevate  me  here,', 'here,  At  the  corner  of  your  lips  As  the  orbit  of  your  hips  Eclipse,  you  elevate  my', "my  soul   I've  lost  all  self-control  Been  living  like  a  mole  Now  going  down,", 'down,  excavation  I  and  I  in  the  sky  You  make  me  feel  like  I  can  fly  So  high,', 'high,  elevation   A  star  lit  up  like  a  cigar  Strung  out  like  a  guitar  Maybe  you', "you  could  educate  my  mind  Explain  all  these  controls  I  can't  sing  but  I've  got  soul", 'got  soul  The  goal  is  elevation   A  mole,  living  in  a  hole']
High,  higher  than  the  sun  You  shoot  me  from  a  gun  I  need  you  to  elevate  me  here,
here,  At  the  corner  of  your  lips  As  the  orbit  of  your  hips  Eclipse,  you  elevate  my
my  soul   I've  lost  all  self-control  Been  living  like  a  mole  Now  going  down,
down,  excavation  I  and  I  in  the  sky  You  ma

7

## Semantic text splitter/ experiment

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

llm = OpenAIEmbeddings()

# about_loader is defined in another cell above. used Pydfloader to load PDF doc from gdrive.
# about_loader = PyPDFLoader(file_path)

about_rodi_docs = about_rodi_loader.load()
print("dasdasdasd")
print(len(about_rodi_docs))
print(about_rodi_docs)

splitter = SemanticChunker(
    embeddings = llm,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=40
)

docs = splitter.create_documents([about_pdf_docs[0].page_content])
print(docs)


dasdasdasd
1
[Document(metadata={'producer': 'Skia/PDF m142 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': '/content/drive/MyDrive/Projects/docs/rodi.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Rohit  is  a  software  engineer.  For  work,  he  automates  complex  business  workflows  and  \nbuilds\n \nAI\n \nsystems.\n \nHe\n \nhas\n \nbeen\n \nworking\n \nfor\n \nover\n \n15\n \nyears\n \nin\n \nthe\n \ntech\n \nindustry.\n \nHe\n \nhas\n \ntaken\n \nvarious\n \nroles,\n \nhis\n \nlatest\n \nrole\n \nbeing\n \nof\n \na\n \nCloud\n \nArchitect.\n \nHe\n \nloves\n \nto\n \nwatch\n \ndrama\n \nmovies\n \nand\n \nlistens\n \nto\n \nmusic.\n \nHe\n \nloves\n \nto\n \ntake\n \na\n \ndrive\n \nand\n \ndo\n \nroad\n \ntrips\n \nwith\n \nfriends\n \nand\n \nfamily,\n \nespecially\n \nin\n \nthe\n \nmonsoons.')]
[Document(metadata={}, page_content='Rohit  is  a  software  engineer. For  work,  he  automates  compl

Quetions
- choosing dimensions for embedding
- chunking size
- try sim search/ default, compress context search, MMR, compare results
- compare perf, text splittint types for better perf?